Evaluate model epoch 4 no embedding

# LSTM model

In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import pickle

scaler_file = "../dataset/processed/scaler.pkl"

with open(scaler_file, 'rb') as f:
    scaler_X, scaler_y = pickle.load(f)

feature_cols = ["Latitude", "Longitude", "hour", "day", "month", "season", 
                "WS10M", "QV2M", "PS", "PRECTOTCORR", "T2M", "CLRSKY_SFC_SW_DWN"]

target_cols = ["CLRSKY_SFC_SW_DWN", "PS", "T2M", "QV2M", "WS10M", "PRECTOTCORR"] 

timesteps = 24
batch_size = 256

test_file = "../dataset/processed/test_data.csv"

checkpoint_path = "../model/best_model2.h5"
model = load_model(checkpoint_path)
model.summary()

def data_generator(file_path, feature_cols, target_cols, batch_size=256, timesteps=24):
    dtype_dict = {col: np.float32 for col in feature_cols + target_cols}
    dtype_dict.update({"hour": np.int8, "day": np.int8, "month": np.int8, "season": np.int8})

    for chunk in pd.read_csv(file_path, chunksize=batch_size * 5, dtype=dtype_dict, parse_dates=["Datetime"], low_memory=True):
        chunk = chunk.sort_values(by=["Datetime"])
        
        X_scaled = scaler_X.transform(chunk[feature_cols])
        y_scaled = scaler_y.transform(chunk[target_cols])

        X_batch, y_batch = [], []
        for i in range(len(chunk) - timesteps - 24):
            X_batch.append(X_scaled[i:i+timesteps])
            y_batch.append(y_scaled[i+timesteps:i+timesteps+24])

        yield np.array(X_batch), np.array(y_batch)

test_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(test_file, feature_cols, target_cols, batch_size, timesteps),
    output_signature=(
        tf.TensorSpec(shape=(None, timesteps, len(feature_cols)), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 24, len(target_cols)), dtype=tf.float32),
    )
).prefetch(tf.data.AUTOTUNE)

loss, mae = model.evaluate(test_dataset)
print(f"✅ Kết quả đánh giá trên dữ liệu test: Loss={loss:.4f}, MAE={mae:.4f}")


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 24, 128)           72192     
                                                                 
 dropout (Dropout)           (None, 24, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 144)               9360      
                                                                 
 reshape (Reshape)           (None, 24, 6)             0         
                                                                 
Total params: 130,960
Trainable params: 130,960
Non-trai

# Fine-tune & CNN-LSTM

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import pickle

scaler_file = "../dataset/processed/scaler.pkl"
with open(scaler_file, 'rb') as f:
    scaler_X, scaler_y = pickle.load(f)

feature_cols = ["Latitude", "Longitude", "hour", "day", "month", "season", 
                "WS10M", "QV2M", "PS", "PRECTOTCORR", "T2M", "CLRSKY_SFC_SW_DWN"]
target_cols = ["CLRSKY_SFC_SW_DWN", "PS", "T2M", "QV2M", "WS10M", "PRECTOTCORR"] 

timesteps = 24
batch_size = 256

test_file = "../dataset/processed/test_data.csv"
checkpoint_path = "best_model_spatial.h5"
model = load_model(checkpoint_path)

print("Model: \"sequential\"")
model.summary()

def data_generator(file_path, feature_cols, target_cols, batch_size=256, timesteps=24):
    dtype_dict = {col: np.float32 for col in feature_cols + target_cols}
    dtype_dict.update({"hour": np.int8, "day": np.int8, "month": np.int8, "season": np.int8})

    for chunk in pd.read_csv(file_path, chunksize=batch_size * 5, dtype=dtype_dict, parse_dates=["Datetime"], low_memory=True):
        chunk = chunk.sort_values(by=["Datetime", "Latitude", "Longitude"])
        
        X_scaled = scaler_X.transform(chunk[feature_cols])
        y_scaled = scaler_y.transform(chunk[target_cols])

        X_batch, y_batch = [], []
        for i in range(len(chunk) - timesteps - 24):
            X_batch.append(X_scaled[i:i+timesteps])
            y_batch.append(y_scaled[i+timesteps:i+timesteps+24])

        yield np.array(X_batch), np.array(y_batch)

test_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(test_file, feature_cols, target_cols, batch_size, timesteps),
    output_signature=(
        tf.TensorSpec(shape=(None, timesteps, len(feature_cols)), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 24, len(target_cols)), dtype=tf.float32),
    )
).prefetch(tf.data.AUTOTUNE)

print("\nĐang đánh giá mô hình trên tập test...")
loss, mae = model.evaluate(test_dataset, verbose=1)
print(f"✅ Kết quả đánh giá trên dữ liệu test: Loss={loss:.4f}, MAE={mae:.4f}")

Model: "sequential"
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 24, 128)           72192     
                                                                 
 dropout (Dropout)           (None, 24, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 144)               9360      
                                                                 
 reshape (Reshape)           (None, 24, 6)             0         
                                                                 
Total params: 130,960
Trainable para

# Fine-tune LSTM

In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import pickle

scaler_file = "../dataset/processed/scaler.pkl"
with open(scaler_file, 'rb') as f:
    scaler_X, scaler_y = pickle.load(f)

feature_cols = ["Latitude", "Longitude", "hour", "day", "month", "season", 
                "WS10M", "QV2M", "PS", "PRECTOTCORR", "T2M", "CLRSKY_SFC_SW_DWN"]
target_cols = ["CLRSKY_SFC_SW_DWN", "PS", "T2M", "QV2M", "WS10M", "PRECTOTCORR"] 

timesteps = 24
batch_size = 256

test_file = "../dataset/processed/test_data.csv"
checkpoint_path = "best_model_spatial.h5"
model = load_model(checkpoint_path)

print("Model: \"sequential\"")
model.summary()

def data_generator(file_path, feature_cols, target_cols, batch_size=256, timesteps=24):
    dtype_dict = {col: np.float32 for col in feature_cols + target_cols}
    dtype_dict.update({"hour": np.int8, "day": np.int8, "month": np.int8, "season": np.int8})

    for chunk in pd.read_csv(file_path, chunksize=batch_size * 5, dtype=dtype_dict, parse_dates=["Datetime"], low_memory=True):
        chunk = chunk.sort_values(by=["Datetime", "Latitude", "Longitude"])
        
        X_scaled = scaler_X.transform(chunk[feature_cols])
        y_scaled = scaler_y.transform(chunk[target_cols])

        X_batch, y_batch = [], []
        for i in range(len(chunk) - timesteps - 24):
            X_batch.append(X_scaled[i:i+timesteps])
            y_batch.append(y_scaled[i+timesteps:i+timesteps+24])

        yield np.array(X_batch), np.array(y_batch)

test_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(test_file, feature_cols, target_cols, batch_size, timesteps),
    output_signature=(
        tf.TensorSpec(shape=(None, timesteps, len(feature_cols)), dtype=tf.float32), 
        tf.TensorSpec(shape=(None, 24, len(target_cols)), dtype=tf.float32),
    )
).prefetch(tf.data.AUTOTUNE)

print("\nĐang đánh giá mô hình trên tập test...")
loss, mae = model.evaluate(test_dataset, verbose=1)
print(f"✅ Kết quả đánh giá trên dữ liệu test: Loss={loss:.4f}, MAE={mae:.4f}")

Model: "sequential"
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 24, 128)           72192     
                                                                 
 dropout (Dropout)           (None, 24, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 144)               9360      
                                                                 
 reshape (Reshape)           (None, 24, 6)             0         
                                                                 
Total params: 130,960
Trainable para

In [26]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

# ========================== LOAD & CLEAN DATA ==========================

def load_processed_data():
    df_day_20 = pd.read_csv('../dataset/weather_data_25.csv')
    df_day_21 = pd.read_csv('../dataset/weather_data_26.csv')
    return df_day_20, df_day_21

def add_time_features(df):
    df['Datetime'] = pd.to_datetime(df['Datetime'], errors='coerce')
    df['hour'] = df['Datetime'].dt.hour
    df['day'] = df['Datetime'].dt.day
    df['month'] = df['Datetime'].dt.month
    df['season'] = (df['month'] % 12 + 3) // 3
    return df

# ========================== LSTM INPUT ==========================

def create_lstm_input(df, time_steps=24):
    X, y = [], []
    features = ["Latitude", "Longitude", "hour", "day", "month", "season", 
                "WS10M", "QV2M", "PS", "PRECTOTCORR", "T2M", "CLRSKY_SFC_SW_DWN"]

    for i in range(len(df) - time_steps - 24 + 1):
        X_window = df.iloc[i:i+time_steps][features].values
        y_window = df.iloc[i+time_steps:i+time_steps+24][features[:6]].values
        if y_window.shape[0] == 24:
            X.append(X_window)
            y.append(y_window)
    return np.array(X), np.array(y)

# ========================== FORECAST ==========================

def predict_with_model_for_lat_lon(df_day_20, lat, lon, scaler_X, scaler_y, model):
    df_lat_lon = df_day_20[(df_day_20['Latitude'] == lat) & (df_day_20['Longitude'] == lon)].copy()
    if len(df_lat_lon) == 0:
        print(f"No data for lat {lat}, lon {lon}")
        return None, None

    # Thêm cột thời gian
    df_lat_lon = add_time_features(df_lat_lon)
    
    # Đảm bảo cột PRECTOTCORR có mặt
    df_lat_lon = reorder_and_clean_columns(df_lat_lon)
    
    X_input, _ = create_lstm_input(df_lat_lon, time_steps=24)
    if X_input.shape[0] == 0:
        print(f"Not enough data for prediction at lat {lat}, lon {lon}")
        return None, None

    features = ["Latitude", "Longitude", "hour", "day", "month", "season", 
                "WS10M", "QV2M", "PS", "PRECTOTCORR", "T2M", "CLRSKY_SFC_SW_DWN"]

    X_input_df = pd.DataFrame(X_input.reshape(-1, len(features)), columns=features)
    X_input_df = X_input_df[scaler_X.feature_names_in_]
    X_input_scaled = scaler_X.transform(X_input_df).reshape(X_input.shape)

    y_pred_scaled = model.predict(X_input_scaled)
    y_pred = scaler_y.inverse_transform(y_pred_scaled[-1])  # 24 dòng cuối

    forecast_start = df_lat_lon['Datetime'].iloc[-1] + pd.Timedelta(hours=1)
    forecast_times = pd.date_range(start=forecast_start, periods=24, freq='h')

    forecast_df = pd.DataFrame(y_pred, columns=[
        'Predicted_T2M', 'Predicted_QV2M', 'Predicted_PS', 'Predicted_WS10M',
        'Predicted_PRECTOTCORR', 'Predicted_CLRSKY_SFC_SW_DWN'
    ])
    forecast_df['Datetime'] = forecast_times
    forecast_df['Latitude'] = lat
    forecast_df['Longitude'] = lon

    return forecast_df, y_pred

def forecast_for_all_lat_lon(df_day_20, scaler_X, scaler_y):
    model = load_model('best_model_fine_tune.h5')
    available_coords = df_day_20[['Latitude', 'Longitude']].drop_duplicates().values
    forecast_results_list = []

    for lat, lon in available_coords:
        forecast_df, _ = predict_with_model_for_lat_lon(df_day_20, lat, lon, scaler_X, scaler_y, model)
        if forecast_df is not None:
            forecast_results_list.append(forecast_df)

    forecast_results = pd.concat(forecast_results_list, ignore_index=True)
    return forecast_results

# ========================== EVALUATE ==========================

def evaluate_model(df_day_21, forecast_results):
    df_day_21['Datetime'] = pd.to_datetime(df_day_21['Datetime'], errors='coerce')
    forecast_results['Datetime'] = pd.to_datetime(forecast_results['Datetime'], errors='coerce')

    df_day_21['Latitude'] = df_day_21['Latitude'].astype(np.float32)
    df_day_21['Longitude'] = df_day_21['Longitude'].astype(np.float32)
    forecast_results['Latitude'] = forecast_results['Latitude'].astype(np.float32)
    forecast_results['Longitude'] = forecast_results['Longitude'].astype(np.float32)

    merged = pd.merge(df_day_21, forecast_results,
                      on=['Datetime', 'Latitude', 'Longitude'],
                      how='inner', suffixes=('', '_pred'))

    if len(merged) == 0:
        print("No matching data for evaluation")
        return

    print(f"\n✅ Số lượng dòng được so sánh: {len(merged)}")
    print(merged[['Datetime', 'Latitude', 'Longitude'] + [f'Predicted_{c}' for c in ['T2M', 'QV2M']]].head())

    metrics = {}
    for col in ["CLRSKY_SFC_SW_DWN", "PS", "T2M", "QV2M", "WS10M", "PRECTOTCORR"]:  # Sửa tên cột ở đây
        actual = merged[col]
        pred = merged[f'Predicted_{col}']
        metrics[col] = {
            'MAE': mean_absolute_error(actual, pred),
            'MSE': mean_squared_error(actual, pred),
            'R2': r2_score(actual, pred)
        }

    print("\n🔍 Chi tiết các chỉ số đánh giá:\n")
    df_metrics = pd.DataFrame(metrics).T
    df_metrics.columns = ['MAE', 'MSE', 'R2']
    print(df_metrics.round(4))

# ========================== MAIN ==========================

if __name__ == "__main__":
    df_day_20, df_day_21 = load_processed_data()
    df_day_20_unscaled = df_day_20.copy()

    with open('../dataset/scaler.pkl', 'rb') as f:
        scaler_X, scaler_y = pickle.load(f)

    forecast_results = forecast_for_all_lat_lon(df_day_20_unscaled, scaler_X, scaler_y)

    if len(forecast_results) > 0:
        forecast_results['Latitude'] = forecast_results['Latitude'].round(4)
        forecast_results['Longitude'] = forecast_results['Longitude'].round(4)

        evaluate_model(df_day_21, forecast_results)
        
        forecast_results.to_csv("forecast_day26.csv", index=False)
        print("💾 Đã lưu kết quả dự báo vào forecast_day26.csv")

        print("\n📊 Kết quả dự báo:")
        print(forecast_results.head())
        print("📈 Khoảng thời gian dự báo:",
              forecast_results['Datetime'].min(), "→", forecast_results['Datetime'].max())

        print("\n📊 Dữ liệu thực tế:")
        print(df_day_21.head())
        print("📈 Khoảng thời gian thực tế:",
              df_day_21['Datetime'].min(), "→", df_day_21['Datetime'].max())
    else:
        print("❌ Không có kết quả dự báo để đánh giá.")
        
            # Hiển thị kết quả dự báo và thực tế chi tiết hơn
        print("\n📋 Một vài dòng dữ liệu dự báo:")
        print(forecast_results[['Datetime', 'Latitude', 'Longitude'] + [col for col in forecast_results.columns if col.startswith('Predicted_')]].head(10))

        print("\n📋 Một vài dòng dữ liệu thực tế:")
        print(df_day_21[['Datetime', 'Latitude', 'Longitude', 'T2M', 'QV2M', 'PS', 'WS10M', 'PRECTOTCORR', 'CLRSKY_SFC_SW_DWN']].head(10))

    # Vẽ biểu đồ so sánh
    def plot_results_for_location(df_actual, df_pred, lat, lon):
        actual_data = df_actual[(df_actual['Latitude'] == lat) & (df_actual['Longitude'] == lon)]
        pred_data = df_pred[(df_pred['Latitude'] == lat) & (df_pred['Longitude'] == lon)]

        if actual_data.empty or pred_data.empty:
            print(f"Không có dữ liệu cho toạ độ ({lat}, {lon})")
            return

        cols = ["CLRSKY_SFC_SW_DWN", "PS", "T2M", "QV2M", "WS10M", "PRECTOTCORR"]
        plt.figure(figsize=(18, 12))
        for i, col in enumerate(cols):
            plt.subplot(3, 2, i+1)
            plt.plot(actual_data['Datetime'], actual_data[col], label='Thực tế', color='blue')
            plt.plot(pred_data['Datetime'], pred_data[f'Predicted_{col}'], label='Dự báo', linestyle='--', color='red')
            plt.title(f'{col} tại ({lat}, {lon})')
            plt.xlabel('Thời gian')
            plt.ylabel(col)
            plt.xticks(rotation=45)
            plt.grid(True)
            plt.legend()
        plt.tight_layout()
        plt.show()

    # Chọn một toạ độ phổ biến nhất để vẽ
    most_common_coord = forecast_results.groupby(['Latitude', 'Longitude']).size().idxmax()
    plot_results_for_location(df_day_21, forecast_results, *most_common_coord)


17/17 [==============================] - 0s 6ms/step

✅ Số lượng dòng được so sánh: 166464
    Datetime  Latitude  Longitude  Predicted_T2M  Predicted_QV2M
0 2024-07-26       8.0      102.0     480.932922       68.425316
1 2024-07-26       8.0      102.0     480.932922       68.425316
2 2024-07-26       8.0      102.0     480.932922       68.425316
3 2024-07-26       8.0      102.0     480.932922       68.425316
4 2024-07-26       8.0      102.0     480.932922       68.425316


KeyError: 'PRECTOTCORR'

In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# 1. Cấu hình đường dẫn
scaler_file = "../dataset/processed/scaler.pkl"
train_file = "../dataset/weather_data_25.csv"  # Dùng để dự báo
test_file = "../dataset/weather_data_26.csv"   # Dùng để đánh giá
checkpoint_path = "best_model_fine_tune.h5"

# 2. Các cột đặc trưng và mục tiêu
feature_cols = ["Latitude", "Longitude", "hour", "day", "month", "season", 
               "WS10M", "QV2M", "PS", "PRECTOTCORR", "T2M", "CLRSKY_SFC_SW_DWN"]
target_cols = ["CLRSKY_SFC_SW_DWN", "PS", "T2M", "QV2M", "WS10M", "PRECTOTCORR"]

timesteps = 24
batch_size = 256

# 3. Tải model và scaler
with open(scaler_file, 'rb') as f:
    scaler_X, scaler_y = pickle.load(f)

model = load_model(checkpoint_path)

# 4. Hàm tiền xử lý dữ liệu (cải tiến)
def preprocess_data(df):
    # Chuẩn hóa tên cột
    if 'PRECTOT' in df.columns and 'PRECTOTCORR' not in df.columns:
        df = df.rename(columns={'PRECTOT': 'PRECTOTCORR'})
    
    # Thêm các đặc trưng thời gian
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    df['hour'] = df['Datetime'].dt.hour.astype(np.int8)
    df['day'] = df['Datetime'].dt.day.astype(np.int8)
    df['month'] = df['Datetime'].dt.month.astype(np.int8)
    df['season'] = ((df['month'] % 12 + 3) // 3).astype(np.int8)
    
    # Xử lý dữ liệu số
    for col in feature_cols + target_cols:
        if col in df.columns:
            if df[col].dtype == 'object':
                df[col] = pd.to_numeric(df[col].astype(str).str.extract('([-+]?\d*\.?\d+)')[0], errors='coerce')
            df[col] = df[col].fillna(df[col].mean())
    return df

def evaluation_data_generator(file_path):
    dtype_dict = {col: np.float32 for col in feature_cols + target_cols}
    dtype_dict.update({"hour": np.int8, "day": np.int8, "month": np.int8, "season": np.int8})

    for chunk in pd.read_csv(file_path, chunksize=batch_size*5, dtype=dtype_dict, parse_dates=["Datetime"]):
        chunk = preprocess_data(chunk)
        
        # Kiểm tra nếu chunk đủ dài
        if len(chunk) < timesteps + 24:
            print(f"Warning: Chunk size {len(chunk)} too small for timesteps={timesteps} and forecast=24. Skipping...")
            continue
        
        X_scaled = scaler_X.transform(chunk[feature_cols])
        y_scaled = scaler_y.transform(chunk[target_cols])

        X_batch, y_batch = [], []
        for i in range(len(chunk) - timesteps - 24):
            X_batch.append(X_scaled[i:i+timesteps])
            y_batch.append(y_scaled[i+timesteps:i+timesteps+24])
        
        if X_batch:  # Chỉ yield nếu có dữ liệu
            yield np.array(X_batch), np.array(y_batch)
        else:
            print("Warning: No valid sequences generated from this chunk.")

def debug_scalers(scaler_X, scaler_y):
    print("\n=== DEBUG SCALERS ===")
    print("Scaler_X details:")
    print(f"- Feature names: {scaler_X.feature_names_in_}")
    print(f"- Data min: {scaler_X.data_min_}")
    print(f"- Data max: {scaler_X.data_max_}")
    print(f"- Scale: {scaler_X.scale_}")
    print(f"- Min: {scaler_X.min_}")
    
    print("\nScaler_y details:")
    print(f"- Feature names: {scaler_y.feature_names_in_}")
    print(f"- Data min: {scaler_y.data_min_}")
    print(f"- Data max: {scaler_y.data_max_}")
    print(f"- Scale: {scaler_y.scale_}")
    print(f"- Min: {scaler_y.min_}")

def debug_data_sample(df, name="Data"):
    print(f"\n=== DEBUG {name.upper()} SAMPLE ===")
    print(f"Shape: {df.shape}")
    print("\nFirst 3 rows:")
    print(df.head(3))
    print("\nLast 3 rows:")
    print(df.tail(3))
    print("\nDescriptive stats:")
    print(df.describe())
    print("\nMissing values:")
    print(df.isnull().sum())

def debug_model_predictions(y_true, y_pred, target_names):
    print("\n=== DEBUG PREDICTIONS ===")
    for i, col in enumerate(target_names):
        print(f"\n{col}:")
        print(f"- True mean: {y_true[:,i].mean():.4f}")
        print(f"- Pred mean: {y_pred[:,i].mean():.4f}")
        print(f"- MAE: {np.mean(np.abs(y_true[:,i] - y_pred[:,i])):.4f}")
        print(f"- Max Error: {np.max(np.abs(y_true[:,i] - y_pred[:,i])):.4f}")
        
# 6. Hàm dự báo (từ cách thứ 1 nhưng đã cải tiến)
def make_forecast(model, input_data):
    print("\n=== FORECAST DEBUG START ===")
    
    # Tiền xử lý
    input_data = preprocess_data(input_data)
    debug_data_sample(input_data[feature_cols + target_cols], "Input Data")
    
    # Lấy dữ liệu cuối cùng
    X_df = input_data[feature_cols].iloc[-timesteps:]
    print(f"\nUsing last {timesteps} timesteps for prediction:")
    print(X_df)
    
    # Chuẩn hóa
    print("\nScaling details:")
    print("First row before scaling:", X_df.iloc[0].values)
    X_scaled = scaler_X.transform(X_df)
    print("First row after scaling:", X_scaled[0])
    
    X_scaled = X_scaled.reshape(1, timesteps, len(feature_cols))
    
    # Dự báo
    print("\nMaking prediction...")
    y_pred_scaled = model.predict(X_scaled, verbose=1)
    print("\nRaw scaled predictions:")
    print(y_pred_scaled)
    
    # Inverse transform
    print("\nInverse transforming predictions...")
    y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, len(target_cols)))
    print("First 5 predicted values after inverse transform:")
    print(y_pred[:5])
    
    # Tạo DataFrame kết quả
    forecast_dates = pd.date_range(
        start=input_data['Datetime'].iloc[-1] + pd.Timedelta(hours=1),
        periods=24,
        freq='h'
    )
    
    print("\n=== FORECAST DEBUG END ===")
    return pd.DataFrame({
        'Datetime': forecast_dates,
        **{f'Predicted_{col}': y_pred[:, i] for i, col in enumerate(target_cols)}
    })

# 7. Đánh giá mô hình
def evaluate_model():
    # Tạo dataset
    test_dataset = tf.data.Dataset.from_generator(
        lambda: evaluation_data_generator(test_file),
        output_signature=(
            tf.TensorSpec(shape=(None, timesteps, len(feature_cols)), dtype=tf.float32),
            tf.TensorSpec(shape=(None, 24, len(target_cols)), dtype=tf.float32)
        )
    ).prefetch(tf.data.AUTOTUNE)

    # Đánh giá
    print("\nEvaluating model...")
    loss, mae = model.evaluate(test_dataset, verbose=1)
    print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

    # Dự báo mẫu để debug
    print("\nRunning sample predictions for debugging...")
    for X_batch, y_batch in evaluation_data_generator(test_file):
        # Lấy batch đầu tiên
        X_sample = X_batch[0:1]  # Lấy mẫu đầu tiên
        y_true = y_batch[0:1]   # Giá trị thực tế tương ứng
        
        # Dự báo
        y_pred_scaled = model.predict(X_sample, verbose=0)
        
        # Inverse transform
        y_true = scaler_y.inverse_transform(y_true.reshape(-1, len(target_cols)))
        y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, len(target_cols)))
        
        # Debug
        debug_model_predictions(y_true, y_pred, target_cols)
        break  # Chỉ kiểm tra batch đầu tiên

# 8. So sánh dự báo với thực tế
def compare_results(forecast_df):
    # Đọc và tiền xử lý dữ liệu thực tế
    actual_df = preprocess_data(pd.read_csv(test_file))
    actual_24h = actual_df.iloc[:24].copy()
    
    # Ghép dữ liệu
    comparison = pd.merge(
        forecast_df.rename(columns={f'Predicted_{col}': col for col in target_cols}),
        actual_24h[['Datetime'] + target_cols],
        on='Datetime',
        suffixes=('_pred', '_actual')
    )
    
    # Tính toán sai số
    for col in target_cols:
        comparison[f'{col}_error'] = comparison[f'{col}_actual'] - comparison[f'{col}_pred']
    
    # Visualize
    plt.figure(figsize=(15, 10))
    for i, col in enumerate(target_cols, 1):
        plt.subplot(3, 2, i)
        plt.plot(comparison['Datetime'], comparison[f'{col}_actual'], label='Thực tế')
        plt.plot(comparison['Datetime'], comparison[f'{col}_pred'], label='Dự báo')
        plt.title(col)
        plt.legend()
        plt.grid()
    plt.tight_layout()
    plt.show()
    
    return comparison

# Hàm check_data_distribution sửa lại
def check_data_distribution():
    train_df = pd.read_csv(train_file)
    test_df = pd.read_csv(test_file)
    
    # Tiền xử lý dữ liệu trước khi kiểm tra
    train_df = preprocess_data(train_df)
    test_df = preprocess_data(test_df)
    
    print("\nPhân phối dữ liệu train vs test:")
    for col in feature_cols + target_cols:
        if col in train_df.columns and col in test_df.columns:
            plt.figure(figsize=(10, 4))
            plt.hist(train_df[col], bins=30, alpha=0.5, label='Train')
            plt.hist(test_df[col], bins=30, alpha=0.5, label='Test')
            plt.title(col)
            plt.legend()
            plt.show()
        else:
            print(f"Warning: Column {col} not found in data")

# Hàm main sửa lại
def main():
    try:
        # Kiểm tra scaler chi tiết
        debug_scalers(scaler_X, scaler_y)
        
        # Kiểm tra dữ liệu train/test
        print("\nChecking training data...")
        train_df = pd.read_csv(train_file)
        train_df = preprocess_data(train_df)
        debug_data_sample(train_df, "Training Data")
        
        print("\nChecking test data...")
        test_df = pd.read_csv(test_file)
        test_df = preprocess_data(test_df)
        debug_data_sample(test_df, "Test Data")
        
        # Đánh giá model với debug
        evaluate_model()
        
        # Dự báo với debug chi tiết
        print("\nMaking forecast with debug...")
        forecast_df = make_forecast(model, train_df)
        
        if forecast_df is not None:
            print("\nForecast results:")
            print(forecast_df)
            
            # So sánh với thực tế
            comparison_df = compare_results(forecast_df)
            
            # Tính toán sai số chi tiết
            print("\nError Analysis:")
            for col in target_cols:
                if f'{col}_error' in comparison_df.columns:
                    errors = comparison_df[f'{col}_error']
                    print(f"\n{col}:")
                    print(f"- Mean: {errors.mean():.4f}")
                    print(f"- MAE: {errors.abs().mean():.4f}")
                    print(f"- Max Error: {errors.abs().max():.4f}")
                    print(f"- Std Dev: {errors.std():.4f}")
                    print(f"- Median: {errors.median():.4f}")
                    
                    # Phân phối sai số
                    plt.figure()
                    plt.hist(errors, bins=30)
                    plt.title(f'Error Distribution - {col}')
                    plt.show()
    
    except Exception as e:
        print(f"\n❌ Main Error: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()


=== DEBUG SCALERS ===
Scaler_X details:
- Feature names: ['Latitude' 'Longitude' 'hour' 'day' 'month' 'season' 'WS10M' 'QV2M' 'PS'
 'PRECTOTCORR' 'T2M' 'CLRSKY_SFC_SW_DWN']
- Data min: [   8.  102.    0.    1.    1.    1. -999. -999. -999.    0. -999. -999.]
- Data max: [  24.    118.     23.     31.     12.      4.   1072.65 1063.78 1031.07
 2178.57 1087.62 1073.12]
- Scale: [0.0625     0.0625     0.04347826 0.03333334 0.09090909 0.33333334
 0.00048271 0.00048478 0.00049259 0.00045902 0.00047924 0.0004826 ]
- Min: [-0.5        -6.375       0.         -0.03333334 -0.09090909 -0.33333334
  0.48222435  0.4842979   0.49210125  0.          0.47876468  0.4821149 ]

Scaler_y details:
- Feature names: ['CLRSKY_SFC_SW_DWN' 'PS' 'T2M' 'QV2M' 'WS10M' 'PRECTOTCORR']
- Data min: [-999. -999. -999. -999. -999.    0.]
- Data max: [1073.12 1031.07 1087.62 1063.78 1072.65 2178.57]
- Scale: [0.0004826  0.00049259 0.00047924 0.00048478 0.00048271 0.00045902]
- Min: [0.4821149  0.49210125 0.47876468 0.4

d:\anaconda3\envs\weather_lstm\lib\site-packages\keras\utils\generic_utils.py:993: RuntimeWarning: divide by zero encountered in log10
  numdigits = int(np.log10(self.target)) + 1
Traceback (most recent call last):
  File "C:\Users\tranh\AppData\Local\Temp\ipykernel_24376\458602434.py", line 263, in main
    evaluate_model()
  File "C:\Users\tranh\AppData\Local\Temp\ipykernel_24376\458602434.py", line 171, in evaluate_model
    loss, mae = model.evaluate(test_dataset, verbose=1)
  File "d:\anaconda3\envs\weather_lstm\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "d:\anaconda3\envs\weather_lstm\lib\site-packages\keras\utils\generic_utils.py", line 993, in update
    numdigits = int(np.log10(self.target)) + 1
OverflowError: cannot convert float infinity to integer


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import pickle

# Đường dẫn tới scaler và mô hình đã lưu
scaler_file = "../dataset/processed/scaler.pkl"
with open(scaler_file, 'rb') as f:
    scaler_X, scaler_y = pickle.load(f)

# Các cột đặc trưng và mục tiêu
feature_cols = ["Latitude", "Longitude", "hour", "day", "month", "season", 
                "WS10M", "QV2M", "PS", "PRECTOTCORR", "T2M", "CLRSKY_SFC_SW_DWN"]
target_cols = ["CLRSKY_SFC_SW_DWN", "PS", "T2M", "QV2M", "WS10M", "PRECTOTCORR"] 

# Thời gian chuỗi
timesteps = 24
batch_size = 256

# Đường dẫn tới file test và mô hình đã huấn luyện
test_file = "../dataset/weather_data_26.csv"
checkpoint_path = "best_model_fine_tune.h5"
model = load_model(checkpoint_path)

# Hiển thị mô hình
print("Model: \"sequential\"")
model.summary()

def data_generator(file_path, feature_cols, target_cols, batch_size=256, timesteps=24):
    dtype_dict = {col: np.float32 for col in feature_cols + target_cols}
    dtype_dict.update({"hour": np.int8, "day": np.int8, "month": np.int8, "season": np.int8})

    for chunk in pd.read_csv(file_path, chunksize=batch_size * 5, dtype=dtype_dict, parse_dates=["Datetime"], low_memory=True):
        # Thêm các cột thời gian
        chunk = add_time_features(chunk)  # Gọi hàm để thêm các cột hour, day, month, season
        
        # Đảm bảo cột PRECTOTCORR có mặt
        chunk = reorder_and_clean_columns(chunk)  # Đảm bảo PRECTOTCORR được thêm vào
        
        # Tiến hành chuẩn hóa dữ liệu
        X_scaled = scaler_X.transform(chunk[feature_cols])
        y_scaled = scaler_y.transform(chunk[target_cols])

        X_batch, y_batch = [], []
        for i in range(len(chunk) - timesteps - 24):
            X_batch.append(X_scaled[i:i+timesteps])
            y_batch.append(y_scaled[i+timesteps:i+timesteps+24])

        yield np.array(X_batch), np.array(y_batch)

# Tạo dataset từ generator
test_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(test_file, feature_cols, target_cols, batch_size, timesteps),
    output_signature=(
        tf.TensorSpec(shape=(None, timesteps, len(feature_cols)), dtype=tf.float32),  # Sửa: Sử dụng None thay cho batch_size
        tf.TensorSpec(shape=(None, 24, len(target_cols)), dtype=tf.float32),  # Sửa: Sử dụng None thay cho batch_size
    )
).prefetch(tf.data.AUTOTUNE)

# Kiểm tra nếu có dữ liệu hợp lệ
try:
    for X_batch, y_batch in data_generator(test_file, feature_cols, target_cols, batch_size, timesteps):
        print(f"X_batch shape: {X_batch.shape}, y_batch shape: {y_batch.shape}")
        break  # Chỉ kiểm tra một batch
except Exception as e:
    print(f"Error during data generation: {e}")

# Đánh giá mô hình trên tập test
try:
    print("\nĐang đánh giá mô hình trên tập test...")
    loss, mae = model.evaluate(test_dataset, verbose=1)
    print(f"✅ Kết quả đánh giá trên dữ liệu test: Loss={loss:.4f}, MAE={mae:.4f}")
except Exception as e:
    print(f"Error during model evaluation: {e}")


Model: "sequential"
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 24, 128)           72192     
                                                                 
 dropout (Dropout)           (None, 24, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 144)               9360      
                                                                 
 reshape (Reshape)           (None, 24, 6)             0         
                                                                 
Total params: 130,960
Trainable para

Traceback (most recent call last):
  File "C:\Users\tranh\AppData\Local\Temp\ipykernel_24060\2475724206.py", line 95, in <module>
    print("Sample X values:", X_batch[0, 0, :5])  # In 5 giá trị đầu tiên của mẫu đầu tiên
IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed
